# Pre-requirements


# Libraries and some funtion defination

In [27]:
#!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# Bibliotecas Auxiliares
import seaborn as sns
import numpy as np
import collections
import matplotlib.pyplot as plt
import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff
np.random.seed(0)
from collections import Counter

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras
from scipy import stats


import collections
import functools
import os
import time
import datetime

from sklearn.model_selection import train_test_split

# library to ignore the warnings
import warnings
warnings.filterwarnings("ignore")

print(tf.__version__)

2.12.0


In [28]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    results['matrix'] = np.array(matrix,dtype=object)
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    array.append(np.array(matrix,dtype=object))
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]

def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

# Data Loading and Analysis

In this setion, we load analyse the data and do the transformations needed to prepare the data. As neural networks need numeric values, we must always be sure that all values are numbers.

In [29]:
# input folder
#inputFolders = "../02-transformed-data-new-testes/dados2019/"
inputFolders = "../data_2019_processed/"
            
allFolders = ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
            '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
            '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
            '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
            'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
            'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
            'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
            'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
            'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
            #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
            'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
            'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
            'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
            'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
            'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
            'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
            'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
            #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
            'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
            'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
            'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
            'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4', 
            'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
            'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
            'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
            'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
            'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
            'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']


# client datasets used on the training process (75% of data)
trainFolders =  ['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                '0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                '2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                '2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                '7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                #'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                #'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                #'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4']
                #'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                #'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                #'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                #'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                #'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                #'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']
            
# client datasets used on the training process (25% of data)
testFolders =  [#'0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs',
                #'0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA', 
                #'2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0', 
                #'2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys', 
                #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
                #'7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA', 
                #'a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4', 
                #'ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc', 
                #'Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U', 
                #'CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA', 
                #'DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc', 
                #'DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA', # does not have asleep data
                #'dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY', 
                #'HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo', 
                #'jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw', 
                #'JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I', 
                #'K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8', 
                #'oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q', 
                #'pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM', 
                'PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc', # does not have asleep data
                #'QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ', 
                'rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw', 
                'RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI', 
                #'SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'] 
                'VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is', 
                'Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw', 
                'XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA', 
                'YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw', 
                'ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM', 
                'ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']


In [30]:
print("Preparing test data")
# take the list of directories and concat them
def loadTestData(trainingDataSet):
    print(len(trainingDataSet), "datasets")
    for i in range(0,len(trainingDataSet)):
        print(i , "-", trainingDataSet[i])
        #print(trainingDataSet[i])
        if(i == 0):
            X_test = pd.read_csv(inputFolders+"student_"+trainingDataSet[i]+"_transformed.csv")
        else:
            dataset = pd.read_csv(inputFolders+"student_"+trainingDataSet[i]+"_transformed.csv")
            X_test = pd.concat([X_test, dataset])
    # return the dataset        
    return X_test
         
# test data comprising 25% of the data. It must be fixed to all models being evaluated
X_test = loadTestData(testFolders)

# undestand the dataset by looking on their infos
print(X_test.info())

X_test

Preparing test data
9 datasets
0 - PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc
1 - rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw
2 - RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI
3 - VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is
4 - Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw
5 - XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA
6 - YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw
7 - ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM
8 - ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY
<class 'pandas.core.frame.DataFrame'>
Int64Index: 136286 entries, 0 to 23751
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float64
 1   location            136286 non-null  float64
 2   timestamp           136286 non-null  float64
 3   time_to_next_alarm  136286 non-null  float64
 4   sound               136286 non-null  float64
 5   proximity           136286 non-null  float64
 6   phone_lock          13628

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,1.0,0.000000e+00,0.906151,0.655565,1.0,0.0,0.000000,1.000000,0.115358,2018-05-15 02:46:57,awake
1,0.25,1.0,0.000000e+00,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:27,awake
2,0.25,1.0,3.211588e-07,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:57,awake
3,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:28,awake
4,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:57,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
23747,0.25,1.0,5.819100e-03,0.000099,0.000000,1.0,1.0,0.000236,0.166667,0.510076,2018-06-13 12:14:37,awake
23748,0.25,1.0,5.819743e-03,0.000694,0.000000,1.0,1.0,0.000325,0.166667,0.512856,2018-06-13 12:18:08,awake
23749,0.25,1.0,5.819743e-03,0.000595,0.000000,1.0,1.0,0.000325,0.166667,0.513551,2018-06-13 12:19:08,awake
23750,0.25,1.0,5.820064e-03,0.000595,0.000000,1.0,1.0,0.000354,0.166667,0.513551,2018-06-13 12:19:38,awake


In [31]:
# load cliend data to train
clientList = []

for i in range(0,len(trainFolders)):
    print (i," ", str(inputFolders)+"student_"+trainFolders[i]+"_transformed.csv") #_numeric
    # load client data
    dataset = pd.read_csv(inputFolders+"student_"+trainFolders[i]+"_transformed.csv")
    
    # print(dataset)
    y_train = dataset['class'].copy()
    
    # does not add datasets that dont have instances from both classes
    if y_train.sum() != 0 and (y_train.sum() != len(y_train)):
        clientList.append(dataset)
        
print("Total",(len(clientList)))

0   ../data_2019_processed/student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs_transformed.csv
1   ../data_2019_processed/student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA_transformed.csv
2   ../data_2019_processed/student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0_transformed.csv
3   ../data_2019_processed/student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys_transformed.csv
4   ../data_2019_processed/student_7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA_transformed.csv
5   ../data_2019_processed/student_a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4_transformed.csv
6   ../data_2019_processed/student_ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc_transformed.csv
7   ../data_2019_processed/student_Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U_transformed.csv
8   ../data_2019_processed/student_CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA_transformed.csv
9   ../data_2019_processed/student_DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc_transformed.csv
10   ../data_2019_processed/student_dQEFscjqnIlug8Tgq97JohhS

# Pre-processing

As we saw in the previous section, our dataset has 12 attributes, which we must prepare for the preprocessing task.

In order to do it, we are going to apply the following methodology:
- Feature Selection
- Data encoding transformation
- Feature data type transformation

From our dataset, we need to select the inputs and the outputs from our supervised learning process. From all 12 features, the class is the output, and the input does not need the timestamp and timestamp_text features because they are time-related value and (usually) does not give benefits to the neural network learning process. Thus, we create two new lists of values with the output classes we want: *inputFeatures* and *outputClasses*.

As we are using a process of classification and our output (class) can get two values awake and asleep, we can choose between two approaches:
(1) convert the class using label encoding (0 to awake and 1 to asleep). In this scenario, we use Binary Cross Entropy.
(2) convert the class using one-hot encoding, which becomes two different attributes. In this scenario, we use Categorical Cross Entropy.

In federated learning, I judge easier to use one-hot encoding in such a situation.

*Curiosity*: If you want to check the influence of timestamp in your dataset, you can use the pearson correlation or compare the training process results with and without it.

In [32]:
# selected features
inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]


In [33]:
# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the input dataset
for i in range(0,len(clientList)):
    clientList[i] = transform_output_nominal_class_into_one_hot_encoding(clientList[i])
    #print (clientList[i])

In [34]:
X_test

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class,awake,asleep
0,0.00,1.0,0.000000e+00,0.906151,0.655565,1.0,0.0,0.000000,1.000000,0.115358,2018-05-15 02:46:57,awake,1,0
1,0.25,1.0,0.000000e+00,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:27,awake,1,0
2,0.25,1.0,3.211588e-07,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:57,awake,1,0
3,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:28,awake,1,0
4,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:57,awake,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23747,0.25,1.0,5.819100e-03,0.000099,0.000000,1.0,1.0,0.000236,0.166667,0.510076,2018-06-13 12:14:37,awake,1,0
23748,0.25,1.0,5.819743e-03,0.000694,0.000000,1.0,1.0,0.000325,0.166667,0.512856,2018-06-13 12:18:08,awake,1,0
23749,0.25,1.0,5.819743e-03,0.000595,0.000000,1.0,1.0,0.000325,0.166667,0.513551,2018-06-13 12:19:08,awake,1,0
23750,0.25,1.0,5.820064e-03,0.000595,0.000000,1.0,1.0,0.000354,0.166667,0.513551,2018-06-13 12:19:38,awake,1,0


As FedAvg uses float of 32 bits I recommend convert all values numeric values to float32


In [35]:
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# transforms the data
X_test = transform_data_type(X_test)

X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136286 entries, 0 to 23751
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            136286 non-null  float32
 1   location            136286 non-null  float32
 2   timestamp           136286 non-null  float64
 3   time_to_next_alarm  136286 non-null  float32
 4   sound               136286 non-null  float32
 5   proximity           136286 non-null  float32
 6   phone_lock          136286 non-null  float32
 7   light               136286 non-null  float32
 8   day_of_week         136286 non-null  float32
 9   minutes_day         136286 non-null  float32
 10  timestamp_text      136286 non-null  object 
 11  class               136286 non-null  object 
 12  awake               136286 non-null  float32
 13  asleep              136286 non-null  float32
dtypes: float32(11), float64(1), object(2)
memory usage: 9.9+ MB


In [36]:
X_test

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class,awake,asleep
0,0.00,1.0,0.000000e+00,0.906151,0.655565,1.0,0.0,0.000000,1.000000,0.115358,2018-05-15 02:46:57,awake,1.0,0.0
1,0.25,1.0,0.000000e+00,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:27,awake,1.0,0.0
2,0.25,1.0,3.211588e-07,0.906052,0.567296,1.0,0.0,0.000000,1.000000,0.116053,2018-05-15 02:47:57,awake,1.0,0.0
3,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:28,awake,1.0,0.0
4,0.25,1.0,3.211588e-07,0.905952,0.660604,1.0,0.0,0.000000,1.000000,0.116748,2018-05-15 02:48:57,awake,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23747,0.25,1.0,5.819100e-03,0.000099,0.000000,1.0,1.0,0.000236,0.166667,0.510076,2018-06-13 12:14:37,awake,1.0,0.0
23748,0.25,1.0,5.819743e-03,0.000694,0.000000,1.0,1.0,0.000325,0.166667,0.512856,2018-06-13 12:18:08,awake,1.0,0.0
23749,0.25,1.0,5.819743e-03,0.000595,0.000000,1.0,1.0,0.000325,0.166667,0.513551,2018-06-13 12:19:08,awake,1.0,0.0
23750,0.25,1.0,5.820064e-03,0.000595,0.000000,1.0,1.0,0.000354,0.166667,0.513551,2018-06-13 12:19:38,awake,1.0,0.0


Now I need to transform the data into Tensor Slices. These slices can consider configurations like batch_size, window, and epochs.

The RNNs process data sequences, so we need to specify two properties: (1) the size of the windows; (2) the steps between the beginning of each window.

In [37]:
# General configuration
NUM_EPOCHS = 3
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

In [38]:
# selects the data to train and test
X_test_data = X_test[inputFeatures]
y_test_label = X_test[outputClasses]

# transform to time series
X_test_data, y_test_label = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_label, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

# transtorm data to tensor slices
client_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_label))

#client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE, drop_remainder=True)
client_test_dataset = client_test_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)

print(client_test_dataset.element_spec)
client_test_dataset

(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))


<_BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [39]:
federated_training_data = []
# transform the data
for i in range(0,len(clientList)):
    # selects the data to train and test
    data   = clientList[i][inputFeatures]
    labels = clientList[i][outputClasses]
    # transform data to float32
    #data = transform_data_type(data)

    # transform to time series
    data, labels = create_dataset_time_series_with_one_output(    #timestamp
        data, 
        labels, 
        TIME_SERIES_SIZE, 
        TIME_STEP_SHIFT
    )

    # transform the data to tensor slices
    client_train_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    #client_train_dataset = tf.data.Dataset.from_tensor_slices((data.values, labels.values))
    # apply the configs
    client_train_dataset = client_train_dataset.repeat(NUM_EPOCHS).batch(BATCH_SIZE)
    # transform the data to
    federated_training_data.append(client_train_dataset)

# Model processing

In [40]:
def create_keras_model():
    model = keras.Sequential()
    model.add(LSTM(128,input_shape=(4, 9,))) #return_sequences=True, 
    #model.add(LSTM(128,input_shape=(2, 9,)))
    #model.add(keras.layers.Dropout(rate=0.5))
    #model.add(LSTM(64,activation="tanh"))
    #model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(2, activation='softmax'))#softmax,sigmoid
    
    return model

In [41]:
keras_model = create_keras_model()
#keras_model.summary()
keras_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 128)               70656     
                                                                 
 dense_15 (Dense)            (None, 2)                 258       
                                                                 
Total params: 70,914
Trainable params: 70,914
Non-trainable params: 0
_________________________________________________________________


2023-06-06 20:15:28.988052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:15:28.989779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:15:28.990994: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [42]:
def model_fn():
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    #keras_model = create_keras_model()
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model( #tff.learning.from_keras_model
      keras_model,
      input_spec=client_train_dataset.element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(), #BinaryCrossentropy
      metrics=[tf.keras.metrics.CategoricalAccuracy()]) #, tf.keras.metrics.Precision()

In [43]:
fed_avg_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),#client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

2023-06-06 20:15:29.789270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'batch_input' with dtype double and shape [?,4,9]
	 [[{{node batch_input}}]]
2023-06-06 20:15:29.901290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_1' with dtype double and shape [?,4,9]
	 [[{{node args_1}}]]
2023-06-06 20:15:30.145214: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3' wit

In [44]:
print(fed_avg_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,512],
      float32[128,512],
      float32[512],
      float32[128,2],
      float32[2]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[9,512],
    float32[128,512],
    float32[512],
    float32[128,2],
    float32[2]
  >
>@SERVER)


In [45]:
state = fed_avg_process.initialize()

2023-06-06 20:15:30.818401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 20:15:30.818497: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-06-06 20:15:30.818714: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-06 20:15:30.819458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 20:15:30.819476: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

# First Iteractions (rounds)

Now, we can start the learning process by selecting the users participating in the fed-avg aggregation process. Round 1 uses the data from the first ten users ([0:10]), and Round 2 uses the nine [10:19] remaining users for training.

We must highlight that it is an example. Nineteen users are too few to separate them, but it would be different if we considered much more users or data. In such scenarios, we must determine the selection strategy.

In [46]:
# now we can start the learning process by selecting the users that will participate from the learning process
result = fed_avg_process.next(state, federated_training_data[0:10])
state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

2023-06-06 20:15:31.227556: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 20:15:31.227608: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-06-06 20:15:31.227720: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-06-06 20:15:31.228387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 20:15:31.228404: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8705518), ('precision', 0.8691327), ('loss', 0.3212354), ('num_examples', 572901), ('num_batches', 17908)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


After the aggregation process is finished, we can see the results by loading the neural network weights from the global model.

In [47]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()

keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

# get neural network weights from the global model
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)

# predict using the test data
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX,"\n")
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)

# as we deal with a classification problem with one hot encoding, we must round the values to 0 and 1.
yhat_probs_rounded = yhat_probs.round()
print("\n",yhat_probs_rounded)

y_predicted_df = pd.DataFrame(data=yhat_probs_rounded,columns=['awake','asleep']) 
y_test_real_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

# Instantiate the list that will contain the results
listOfMetrics = list()

print('')
print('awake')
res,resA = printMetrics(y_test_real_label['awake'],y_predicted_df['awake'])
listOfMetrics.append(res)
print('')
print('asleep')
res,resA = printMetrics(y_test_real_label['asleep'],y_predicted_df['asleep'])
listOfMetrics.append(res)
print('')
print('Global')
showGlobalMetrics(listOfMetrics)

2023-06-06 20:15:50.793548: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:15:50.795582: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:15:50.796880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  60/4259 [..............................] - ETA: 7s 

2023-06-06 20:15:51.035935: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:15:51.037649: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:15:51.038627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4259/4259 [==============================] - 8s 2ms/step
0.99218786 0.0078121847 0.5 

[[0.9428349  0.0571652 ]
 [0.94442713 0.05557293]
 [0.9414534  0.05854663]
 ...
 [0.88688624 0.1131138 ]
 [0.88702416 0.11297581]
 [0.8871313  0.1128687 ]]

 [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

awake
Accuracy: 0.814744
Precision: 0.814744
Recall: 1.000000
F1 score: 0.897916
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[     0  25247]
 [     0 111035]]

asleep
Accuracy: 0.816432
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[111265      0]
 [ 25017      0]]

Global
2
accuracy:  0.8155882655082842
precision:  0.4073722134984811
recall:  0.5
f1_score:  0.44895821961288546
cohen_kappa_score:  0.0
roc_auc_score:  0.5


[0.8155882655082842, 0.4073722134984811, 0.5, 0.44895821961288546, 0.0, 0.5]

In [48]:
result = fed_avg_process.next(state, federated_training_data[10:19])
state = result.state
metrics = result.metrics
print('round  2, metrics={}'.format(metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8715857), ('precision', 0.87004083), ('loss', 0.315121), ('num_examples', 649227), ('num_batches', 20292)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [49]:
#def keras_evaluate(state, round_num):
    # Take our global model weights and push them back into a Keras model to
    # use its standard `.evaluate()` method.
keras_model = create_keras_model()

keras_model.compile(
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalAccuracy()])

# get neural network weights from the global model
weights = fed_avg_process.get_model_weights(state)
weights.assign_weights_to(keras_model)

# predict using the test data
yhat_probs = keras_model.predict(X_test_data)

maxX = yhat_probs.max()
minX = yhat_probs.min()
avgX = yhat_probs.mean()

print(maxX,minX,avgX,"\n")
print(yhat_probs)
# predict crisp classes for test set deprecated
#printMetrics(y_test,yhat_probs)

# as we deal with a classification problem with one hot encoding, we must round the values to 0 and 1.
yhat_probs_rounded = yhat_probs.round()
print("\n",yhat_probs_rounded)

y_predicted_df = pd.DataFrame(data=yhat_probs_rounded,columns=['awake','asleep']) 
y_test_real_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep']) 

# Instantiate the list that will contain the results
listOfMetrics = list()

print('')
print('awake')
res,resA = printMetrics(y_test_real_label['awake'],y_predicted_df['awake'])
listOfMetrics.append(res)
print('')
print('asleep')
res,resA = printMetrics(y_test_real_label['asleep'],y_predicted_df['asleep'])
listOfMetrics.append(res)
print('')
print('Global')
showGlobalMetrics(listOfMetrics)

2023-06-06 20:16:21.643352: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:16:21.644995: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:16:21.646699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  24/4259 [..............................] - ETA: 9s   

2023-06-06 20:16:21.919680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:16:21.921850: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:16:21.924190: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4259/4259 [==============================] - 10s 2ms/step
0.9982634 0.0017365747 0.5 

[[0.975198   0.02480208]
 [0.97656274 0.02343725]
 [0.97438157 0.02561835]
 ...
 [0.9541895  0.04581055]
 [0.95429313 0.04570688]
 [0.95437557 0.0456244 ]]

 [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]

awake
Accuracy: 0.814744
Precision: 0.814744
Recall: 1.000000
F1 score: 0.897916
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[     0  25247]
 [     0 111035]]

asleep
Accuracy: 0.816432
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[111265      0]
 [ 25017      0]]

Global
2
accuracy:  0.8155882655082842
precision:  0.4073722134984811
recall:  0.5
f1_score:  0.44895821961288546
cohen_kappa_score:  0.0
roc_auc_score:  0.5


[0.8155882655082842, 0.4073722134984811, 0.5, 0.44895821961288546, 0.0, 0.5]

# Example do script to experiments

Bellow we have a example of script that we can use to execute in a experiment to monitoring the metric improvement for each Round/Iteration

In [50]:
roundData = []

columns = ['NN_type','"neurons"','epochs','batch_size','max_iterations','Users',
           'round_iteration','start_time','end_time','round_time_s','round_time_m',
           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
           'TP','FP','FN','TN']

# maximun number the Rounds\Iteractions
MAX_ITERATIONS = 12 # 120
#NUM_EPOCHS
#BATCH_SIZE
NN_type = 'MLP'
UNITS_NUMBER = "16/8"
USER_NUMBER = len(trainFolders)

# real labels to evaluate the sequences
y_test_real_label = pd.DataFrame(data=y_test_label,columns=['awake','asleep'])

generalData = [NN_type,UNITS_NUMBER,NUM_EPOCHS,BATCH_SIZE,MAX_ITERATIONS,USER_NUMBER]

for i in range(0,MAX_ITERATIONS):
    current_time_start = datetime.datetime.now()
    time_stamp_start = current_time_start.timestamp()
    print(i,"Start timestamp:", time_stamp_start,current_time_start)
    
    result = fed_avg_process.next(state, federated_training_data[0:19])
    state = result.state
    metrics = result.metrics
    print('round  {}, metrics={}'.format(i,metrics))

    print('')
    #def keras_evaluate(state, round_num):
        # Take our global model weights and push them back into a Keras model to
        # use its standard `.evaluate()` method.
    keras_model = create_keras_model()


    keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])

    # get neural network weights
    weights = fed_avg_process.get_model_weights(state)
    weights.assign_weights_to(keras_model)


    # predict values
    yhat_probs = keras_model.predict(X_test_data)
    
    # as we deal with a classification problem with one hot encoding, we must round the values to 0 and 1.
    yhat_probs_rounded = yhat_probs.round()
    
    

    # create a dataframe with the predicted data
    y_predicted_df = pd.DataFrame(data=yhat_probs_rounded,columns=['awake','asleep']) 
    
    # generate time metrics
    current_time_end = datetime.datetime.now()
    time_stamp_end = current_time_end.timestamp()
    processing_time_s = (time_stamp_end-time_stamp_start)
    
    # Instantiate the list that will contain the results
    listOfMetrics = list()
    
    # generate general metrics
    rowData = [i,current_time_start,current_time_end,processing_time_s,(processing_time_s)/60]

    print('')
    print('awake')    
    res,resA = printMetrics(y_test_real_label['awake'],y_predicted_df['awake'])
    listOfMetrics.append(res)
   
    #columns = ['NN_type','units','epochs','batch_size','max_iterations',''Users',
    #            round_iteration','start_time','end_time','round_time_s','round_time_m',
    #           'class','accuracy','precision','recall','f1_score','cohen_kappa_score','roc_auc_score','confusion_matrix',
    #           'TP','FP','FN','TN']
    # new data
    classData = np.concatenate((['awake'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    
    print('')
    print('asleep')
    resA = printMetrics(y_test_real_label['asleep'],y_predicted_df['asleep'])
    listOfMetrics.append(res)
    # new data
    classData = np.concatenate((['asleep'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print('Global')
    resA = showGlobalMetrics(listOfMetrics) #return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score
    # new data
    classData = np.concatenate((['avg'], resA))
    classData = np.concatenate((rowData, classData))
    classData = np.concatenate((generalData, classData))
    roundData.append(classData)
    print('')
    print(i,"End timestamp:", time_stamp_end,current_time_end)
    print('')
    print('')

0 Start timestamp: 1686078993.356348 2023-06-06 20:16:33.356348
round  0, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.8756595), ('precision', 0.87419647), ('loss', 0.3075503), ('num_examples', 1222128), ('num_batches', 38200)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])



2023-06-06 20:17:17.130284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:17:17.132654: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:17:17.134487: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  26/4259 [..............................] - ETA: 8s   

2023-06-06 20:17:17.392472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:17:17.394333: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:17:17.395659: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4259/4259 [==============================] - 9s 2ms/step

awake
Accuracy: 0.814744
Precision: 0.814744
Recall: 1.000000
F1 score: 0.897916
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[     0  25247]
 [     0 111035]]

asleep
Accuracy: 0.816432
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000
Cohens kappa: 0.000000
ROC AUC: 0.500000
\Confusion Matrix
[[111265      0]
 [ 25017      0]]

Global
2
accuracy:  0.8147444269969621
precision:  0.8147444269969621
recall:  1.0
f1_score:  0.8979164392257709
cohen_kappa_score:  0.0
roc_auc_score:  0.5

0 End timestamp: 1686079046.804858 2023-06-06 20:17:26.804858


1 Start timestamp: 1686079047.321783 2023-06-06 20:17:27.321783
round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.87436914), ('precision', 0.87290204), ('loss', 0.3082267), ('num_examples', 1222128), ('num_batches', 38200)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_w

...

4259/4259 [==============================] - 10s 2ms/step

awake
Accuracy: 0.814832
Precision: 0.814936
Recall: 0.999766
F1 score: 0.897939
Cohens kappa: 0.002068
ROC AUC: 0.500635
\Confusion Matrix
[[    38  25209]
 [    26 111009]]

asleep
Accuracy: 0.816520
Precision: 0.593750
Recall: 0.001519
F1 score: 0.003030
Cohens kappa: 0.002095
ROC AUC: 0.500643
\Confusion Matrix
[[111239     26]
 [ 24979     38]]

Global
2
accuracy:  0.8148324797111871
precision:  0.8149363520239616
recall:  0.9997658396001261
f1_score:  0.8979385487739279
cohen_kappa_score:  0.002067730162705339
roc_auc_score:  0.5006354844612109

10 End timestamp: 1686079546.265948 2023-06-06 20:25:46.265948


11 Start timestamp: 1686079546.773631 2023-06-06 20:25:46.773631
round  11, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.87271136), ('precision', 0.8712344), ('loss', 0.30958405), ('num_examples', 1222128), ('num_batches', 38200)]))])), ('agg

2023-06-06 20:26:28.669747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:26:28.671874: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:26:28.673188: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

  26/4259 [..............................] - ETA: 8s   

2023-06-06 20:26:28.945233: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 20:26:28.946998: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 20:26:28.948398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4259/4259 [==============================] - 9s 2ms/step

awake
Accuracy: 0.814781
Precision: 0.814950
Recall: 0.999658
F1 score: 0.897903
Cohens kappa: 0.002213
ROC AUC: 0.500680
\Confusion Matrix
[[    43  25204]
 [    38 110997]]

asleep
Accuracy: 0.816469
Precision: 0.530864
Recall: 0.001719
F1 score: 0.003427
Cohens kappa: 0.002244
ROC AUC: 0.500689
\Confusion Matrix
[[111227     38]
 [ 24974     43]]

Global
2
accuracy:  0.8147811156278892
precision:  0.8149499636566545
recall:  0.9996577655694151
f1_score:  0.8979032179779644
cohen_kappa_score:  0.0022131639817309923
roc_auc_score:  0.5006804691117959

11 End timestamp: 1686079598.186251 2023-06-06 20:26:38.186251




In [51]:
dataMetrics = pd.DataFrame(data=roundData,columns=columns) 

dataMetrics

,NN_type,"""neurons""",epochs,batch_size,max_iterations,Users,round_iteration,start_time,end_time,round_time_s,...,precision,recall,f1_score,cohen_kappa_score,roc_auc_score,confusion_matrix,TP,FP,FN,TN
0,MLP,16/8,3,32,12,19,0,2023-06-06 20:16:33.356348,2023-06-06 20:17:26.804858,53.448510,...,0.814744,1.0,0.897916,0.0,0.5,"[[0, 25247], [0, 111035]]",0.0,25247.0,0.0,111035.0
1,MLP,16/8,3,32,12,19,0,2023-06-06 20:16:33.356348,2023-06-06 20:17:26.804858,53.448510,...,"[0.8164321040196064, 0.0, 0.0, 0.0, 0.0, 0.5, ...",None,None,None,None,None,NaN,NaN,NaN,NaN
2,MLP,16/8,3,32,12,19,0,2023-06-06 20:16:33.356348,2023-06-06 20:17:26.804858,53.448510,...,0.8147444269969621,1.0,0.8979164392257709,0.0,0.5,None,NaN,NaN,NaN,NaN
3,MLP,16/8,3,32,12,19,1,2023-06-06 20:17:27.321783,2023-06-06 20:18:15.003006,47.681223,...,0.81475,0.999955,0.897901,0.000056,0.500017,"[[2, 25245], [5, 111030]]",2.0,25245.0,5.0,111030.0
4,MLP,16/8,3,32,12,19,1,2023-06-06 20:17:27.321783,2023-06-06 20:18:15.003006,47.681223,...,"[0.8164100908410502, 0.2857142857142857, 7.994...",None,None,None,None,None,NaN,NaN,NaN,NaN
5,MLP,16/8,3,32,12,19,1,2023-06-06 20:17:27.321783,2023-06-06 20:18:15.003006,47.681223,...,0.8147495872317007,0.9999549691538704,0.8979014192713599,5.569677801109396e-05,0.5000170932433113,None,NaN,NaN,NaN,NaN
6,MLP,16/8,3,32,12,19,2,2023-06-06 20:18:15.503908,2023-06-06 20:19:01.761154,46.257246,...,0.814756,0.999955,0.897905,0.00012,0.500037,"[[3, 25244], [5, 111030]]",3.0,25244.0,5.0,111030.0
7,MLP,16/8,3,32,12,19,2,2023-06-06 20:18:15.503908,2023-06-06 20:19:01.761154,46.257246,...,"[0.8164174285672355, 0.375, 0.0001199184554502...",None,None,None,None,None,NaN,NaN,NaN,NaN
8,MLP,16/8,3,32,12,19,2,2023-06-06 20:18:15.503908,2023-06-06 20:19:01.761154,46.257246,...,0.8147555659920455,0.9999549691538704,0.8979050499577452,0.00012022439864545209,0.5000368975764995,None,NaN,NaN,NaN,NaN
9,MLP,16/8,3,32,12,19,3,2023-06-06 20:19:02.260043,2023-06-06 20:19:55.061479,52.801436,...,0.814766,0.999946,0.897908,0.000235,0.500072,"[[5, 25242], [6, 111029]]",5.0,25242.0,6.0,111029.0


In [26]:
outputMetricFile = "Results_LSTM_unbalanced_"+str(NUM_EPOCHS)+"_epoch_"+str(BATCH_SIZE)
outputMetricFile = outputMetricFile+"_rounds_w_["+str(TIME_SERIES_SIZE)+"|"+str(TIME_STEP_SHIFT)+"]_03.csv"
dataMetrics.to_csv(outputMetricFile, sep=',', encoding='utf-8', index=False)